## Учебный пример - скоркарта

Построим на уже знакомых нам данных скоркарту.

## Подключаем библиотеки

In [ ]:
# подгружаем все нужные пакеты
import pandas as pd
import numpy as np

# игнорируем warnings
import warnings
warnings.filterwarnings("ignore")

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker
%matplotlib inline

# настройка внешнего вида графиков в seaborn
sns.set_context(
    "notebook", 
    font_scale = 1.5,       
    rc = { 
        "figure.figsize" : (30, 30), 
        "axes.titlesize" : 18 
    }
)

from sklearn.model_selection import train_test_split

import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
from sklearn.linear_model import LogisticRegressionCV

## Загружаем данные

In [ ]:
df = pd.read_csv('auto_app.csv', delimiter=";",decimal=".", encoding="windows-1251")

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
y = df['target']

## Очистка и преобразование данных

Сделаем все преобразование как и в прошлый раз. Работа с пустыми значениями, векторизация, кодирование средним.

In [ ]:
iin = df['IIN']
df = df.drop(('IIN'), axis=1)
df = df.drop(df.columns[[0,1]], axis=1)
df = df.drop(('creditNumber'), axis=1)
df = df.drop(('Вид страхования'), axis=1)

In [ ]:
categorical_columns = [c for c in df.columns if df[c].dtype.name == 'object']
numerical_columns   = [c for c in df.columns if df[c].dtype.name != 'object']
print (categorical_columns)
print (numerical_columns)

In [ ]:
(df[categorical_columns].count(axis=0)/df.shape[0])*100

In [ ]:
ll = [c for c in categorical_columns if (df[c].count()/df.shape[0])*100 <70]

In [ ]:
ll

In [ ]:
df = df.drop((ll), axis=1)

In [ ]:
data_describe = df.describe(include=[object])
for c in ['Пол','Гражданство','Резиденство','Образование','Семейное положение',\
          'Адрес фактического проживания совпадает с адресом регистрации?','Отношение к месту проживания', \
          'Филиал', 'СПФ','Канал продаж'
         ]:
    df[c] = df[c].fillna(data_describe[c]['top'])

In [ ]:
df = df.drop((['Должность клиента','Вид деятельности компании/организации']), axis=1)

In [ ]:
data_describe = df.describe(include=[object])
for c in ['Цель кредитования','Условия кредитования','Наименование атосалона',\
          'Схема финансирования','Категория клиента',\
          'Статус занятости','Имеется работа по совместительству?',\
          'Являетесь ли вы лицом, освобожденным от уплаты обязательных пенсионных взносов в НПФ'
         ]:
    df[c] = df[c].fillna(data_describe[c]['top'])

In [ ]:
for c in ['Кредитная история в БВУ (автомат)']:
    df[c] = df[c].fillna('Отсутствует')

In [ ]:
df.loc[df['Агент'].notnull(), 'Агент_new'] = 1

In [ ]:
df.loc[df['Агент'].isnull(), 'Агент_new'] = 0

In [ ]:
df = df.drop((['Агент']), axis=1)

In [ ]:
categorical_columns = [c for c in df.columns if df[c].dtype.name == 'object']
numerical_columns   = [c for c in df.columns if df[c].dtype.name != 'object']

In [ ]:
binary_columns    = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
print (binary_columns, nonbinary_columns)

In [ ]:
for c in binary_columns[0:]:
    top = data_describe[c]['top']
    top_items = df[c] == top
    df.loc[top_items, c] = 1
    df.loc[np.logical_not(top_items), c] = 0

In [ ]:
for c in nonbinary_columns[0:]:
    df[c+'_mean'] = df.groupby([c])['target'].transform('mean')

In [ ]:
df = df.drop((nonbinary_columns), axis=1)

In [ ]:
def filling(df):
    cat_vars = df.select_dtypes(include=[object]).columns
    num_vars = df.select_dtypes(include=[np.number]).columns
    df[cat_vars] = df[cat_vars].fillna('_MISSING_')
    df[num_vars] = df[num_vars].fillna(np.nan)
    return df

def replace_not_frequent(df, cols, perc_min=5, value_to_replace = "0"):
        else_df = pd.DataFrame(columns=['var', 'list'])
        for i in cols:
            if i != 'date_requested' and i != 'credit_id':
                t = df[i].value_counts(normalize=True)
                q = list(t[t.values < perc_min/100].index)
                if q:
                    else_df = else_df.append(pd.DataFrame([[i, q]], columns=['var', 'list']))
                df.loc[df[i].value_counts(normalize=True)[df[i]].values < perc_min/100, i] =value_to_replace
        else_df = else_df.set_index('var')
        return df, else_df

cat_vars = df.select_dtypes(include=[object]).columns
df = filling(df)

df, else_df = replace_not_frequent(df, cat_vars)

df.drop(['target'], axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, stratify=y, random_state=42)

## Монотонный WOE binning признаков

Теперь начинается самой важный этап в скоринге для регресии – необходимо написать WOE-binning для числовых и категориальных переменных.

Монотонный WOE binning признаков:
- Упрощает интерпретацию
- Обрабатывает отсутствующие значения
- Выявляет сложные нелинейные связи
- Преобразование основано на логарифмическом распределении 
- Упрощает обработку выбросов
- Нет необходимости в dummy variables
- Можно устанавливать монотонную зависимость (либо увеличение, либо уменьшение) между независимой и зависимой переменной


![](https://cdn-images-1.medium.com/max/800/1*6Aw782wiyiFtzvK7EOY8CA.png)

In [ ]:
max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [ ]:
final_iv, IV = data_vars(df,y)

In [ ]:
final_iv

**Information Value (IV)** измеряет предсказательную силу признаков.Считается для каждого признака.
![](https://cdn-images-1.medium.com/max/800/1*9Gi0fGyTpxfwM2TpV4GZQQ.png)

Значения Information Value (IV) для определения cutoff:
- <0.02 Бесполезно для предсказания
- 0.02 – 0.1 Слабая
- 0.1 – 0.3 Средняя
- 0.3 – 0.5 Хорошая
- 0.5+ Слишком хорошо, что бы быть правдой



In [ ]:
IV[IV['VAR_NAME']=='Пол']

In [ ]:
pd.options.display.float_format = '{:40,.4f}'.format

In [ ]:
IV.sort_values('IV', ascending=False)

Теперь можно посмотреть на графиках, как переменные разбились по группам и проверить монотонность или возрастает или убывает. 

In [ ]:
def plot_bin(ev, for_excel=False):
    ind = np.arange(len(ev.index)) 
    width = 0.35
    fig, ax1 = plt.subplots(figsize=(10, 7))
    ax2 = ax1.twinx()
    p1 = ax1.bar(ind, ev['NONEVENT'], width, color=(24/254, 192/254, 196/254))
    p2 = ax1.bar(ind, ev['EVENT'], width, bottom=ev['NONEVENT'], color=(246/254, 115/254, 109/254))

    ax1.set_ylabel('Event Distribution', fontsize=15)
    ax2.set_ylabel('WOE', fontsize=15)

    plt.title(list(ev.VAR_NAME)[0], fontsize=20) 
    ax2.plot(ind, ev['WOE'], marker='o', color='blue')
    # Legend
    plt.legend((p2[0], p1[0]), ('bad', 'good'), loc='best', fontsize=10)

    #Set xticklabels
    q = list()
    for i in range(len(ev)):
        try:
            mn = str(round(ev.MIN_VALUE[i], 2))
            mx = str(round(ev.MAX_VALUE[i], 2))
        except:
            mn = str((ev.MIN_VALUE[i]))
            mx = str((ev.MAX_VALUE[i]))
        q.append(mn + '-' + mx)

    plt.xticks(ind, q, rotation='vertical')
    for tick in ax1.get_xticklabels():
        tick.set_rotation(60)
    #plt.savefig('{}.png'.format(ev.VAR_NAME[0]), dpi=500, bbox_inches = 'tight')
    plt.show() 

def plot_all_bins(iv_df):
    for i in [x.replace('WOE_','') for x in X_train.columns]:
        ev = iv_df[iv_df.VAR_NAME==i]
        ev.reset_index(inplace=True)
        plot_bin(ev)    

In [ ]:
plot_all_bins(final_iv)

Все графики монотонно возрастают или убывают. Ручной биниинг не требуется. Ручной бининг нужен для объединения категориий с близкими значениями WOE в одну категорию так, чтобы максимизировать разницу между группами.

In [ ]:
def adjust_binning(df, bins_dict):
    for i in range(len(bins_dict)):
        key = list(bins_dict.keys())[i]
        if type(list(bins_dict.values())[i])==dict:
            df[key] = df[key].map(list(bins_dict.values())[i])
        else:
            #Categories labels
            categories = list()
            for j in range(len(list(bins_dict.values())[i])):
                if j == 0:
                    categories.append('<'+ str(list(bins_dict.values())[i][j]))
                    try:                        
                        categories.append('(' + str(list(bins_dict.values())[i][j]) +'; '+ str(list(bins_dict.values())[i][j+1]) + ']')
                    except:                       
                        categories.append('(' + str(list(bins_dict.values())[i][j]))
                elif j==len(list(bins_dict.values())[i])-1:
                    categories.append(str(list(bins_dict.values())[i][j]) +'>')
                else:
                    categories.append('(' + str(list(bins_dict.values())[i][j]) +'; '+ str(list(bins_dict.values())[i][j+1]) + ']')
            
            values = [df[key].min()] + list(bins_dict.values())[i]  + [df[key].max()]        
            df[key + '_bins'] = pd.cut(df[key], values, include_lowest=True, labels=categories).astype(object).fillna('_MISSING_').astype(str)
            df[key] = df[key + '_bins']#.map(df.groupby(key + '_bins')[key].agg('median'))
            df.drop([key + '_bins'], axis=1, inplace=True)
    return df

#bins_dict = {    
#   'equi_delinquencyDays': [ 200,400,600]
#    'loan_purpose': {'medicine':'1_group',
 #                   'repair':'1_group',
 #                   'helpFriend':'2_group'}
#}
 
#df = adjust_binning(df, bins_dict)

После осуществлялась проверка на корреляцию. Из двух коррелирующих переменных нужно удалить ту, у которой IV меньше. Кат офф по удалению был взят 0.75.

In [ ]:
def delete_correlated_features(df, cut_off=0.75, exclude = []):
    # Create correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    
    # Plotting All correlations
    f, ax = plt.subplots(figsize=(15, 10))
    plt.title('All correlations', fontsize=20)
    sns.heatmap(X_train.corr(), annot=True)
    
    # Plotting highly correlated
    try:
        f, ax = plt.subplots(figsize=(15, 10))
        plt.title('High correlated', fontsize=20)
        sns.heatmap(corr_matrix[(corr_matrix>cut_off) & (corr_matrix!=1)].dropna(axis=0, how='all').dropna(axis=1, how='all'), annot=True, linewidths=.5)
    except:
        print ('No highly correlated features found')
        
    # Find index of feature columns with correlation greater than cut_off
    to_drop = [column for column in upper.columns if any(upper[column] > cut_off)]
    to_drop = [column for column in to_drop if column not in exclude]
    print ('Dropped columns:', to_drop, '\n')
    df2 = df.drop(to_drop, axis=1)
    print ('Features left after correlation check: {}'.format(len(df.columns)-len(to_drop)), '\n')    
   
    print ('Not dropped columns:', list(df2.columns), '\n')
    
    # Plotting final correlations
    f, ax = plt.subplots(figsize=(15, 10))
    plt.title('Final correlations', fontsize=20)
    sns.heatmap(df2.corr(), annot=True)
    plt.show()
    
    return df2 

In [ ]:
df = delete_correlated_features(df)

In [ ]:
df.shape

Помимо отбора по IV мы добавили рекурсивный поиск оптимального количества переменных методом RFE из sklearn.

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

In [ ]:
def RFE_feature_selection(clf_lr, X, y):
    rfecv = RFECV(estimator=clf_lr, step=1, cv=StratifiedKFold(5), verbose=0, scoring='roc_auc')
    rfecv.fit(X, y)

    print("Optimal number of features : %d" % rfecv.n_features_)

    # Plot number of features VS. cross-validation scores
    f, ax = plt.subplots(figsize=(14, 9))
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (nb of correct classifications)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    plt.show()
    mask = rfecv.get_support()
    X = X.ix[:, mask]
    return X

Далее строилась регрессия и оценивались её метрики на кросс-валидации и тестовой выборке. Обычно все смотрят на коэффициент ROC AUC или Gini.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, stratify=y, random_state=42)

In [ ]:
from sklearn.metrics import auc, accuracy_score, roc_auc_score, f1_score, log_loss, classification_report

In [ ]:
import scikitplot as skplt
import eli5

In [ ]:
def plot_score(clf, X_test, y_test, feat_to_show=30, is_normalize=False, cut_off=0.5):
    #cm = confusion_matrix(pd.Series(clf.predict_proba(X_test)[:,1]).apply(lambda x: 1 if x>cut_off else 0), y_test)
    print ('ROC_AUC:  ', round(roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]), 3))
    print ('Gini:     ', round(2*roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]) - 1, 3))
    print ('F1_score: ', round(f1_score(y_test, clf.predict(X_test)), 3))
    print ('Log_loss: ', round(log_loss(y_test, clf.predict(X_test)), 3))
    
    print ('\n')
    print ('Classification_report: \n', classification_report(pd.Series(clf.predict_proba(X_test)[:,1]).apply(lambda x: 1 if x>cut_off else 0), y_test))
    skplt.metrics.plot_confusion_matrix(y_test, pd.Series(clf.predict_proba(X_test)[:,1]).apply(lambda x: 1 if x>cut_off else 0), title="Confusion Matrix",
                    normalize=is_normalize,figsize=(8,8),text_fontsize='large')
    display(eli5.show_weights(clf, top=20, feature_names = list(X_test.columns)))

clf_lr = LogisticRegressionCV(random_state=1, cv=7)
clf_lr.fit(X_train, y_train)

plot_score(clf_lr, X_test, y_test, cut_off=0.5)

In [ ]:
 RFE_feature_selection(clf_lr,X_train, y_train)

Как мы видим на графике – после 3 переменных качество не изменяется, а значит лишние можно удалить. Для регрессии более 15 переменных в скоринге считается плохим тоном, что в большинстве случаев исправляется с помощью RFE. Модель получилась слабой и нам рекомендуют исипользовать только три признака: Запрошенный срок кредита, Стаж работы на последнем месте работы, Прочие расходы, в месяц. Даи веса все нулевые. Нужно подтюнить модель.

## Вычисление скорбаллов

In [ ]:
coefficients = pd.DataFrame({"Feature":X_train.columns,"Coefficients":np.transpose(clf_lr.coef_[0] )}) 

Посмотрим на коээфициенты

In [ ]:
coefficients.head(10)

In [ ]:
clf_lr.intercept_ 

Модель на столько ужансная, что у нас везде коэффиценты нули. Нормально преоброзовать в скорбаллы не получится. А тюнить модель для учебного примера лень=). Поэтому для примера преобразования мы позьмем для признака Возраст заявителя коэффициент равный 0,2

Скорбалл считается так:
Score = (β×WoE+ α/n)×Factor + Offset/n
Where:
- β — logistic regression coefficient for characteristics that contains the given attribute
- α — logistic regression intercept 
- WoE — Weight of Evidence value for the given attribute
- n — number of characteristics included in the model
- Factor, Offset — scaling parameter

The first four parameters have already been calculated is the previous part. The following formulas are used for calculating factor and offset.
- Factor = pdo/Ln(2)
- Offset = Score — (Factor × ln(Odds))

In [ ]:
IV[IV['VAR_NAME']=='Возраст заявителя']

In [ ]:
coefficients[coefficients['Feature']=='Возраст заявителя'].Coefficients

In [ ]:
beta = coefficients[coefficients['Feature']=='Возраст заявителя'].Coefficients.iloc[0]

In [ ]:
alpha = 0

In [ ]:
n = X_train.shape[1]

In [ ]:
import math

In [ ]:
factor = 40/math.log(2) 

In [ ]:
factor

In [ ]:
offset = 600 -57.7 * math.log(72)

In [ ]:
offset

In [ ]:
final_iv[final_iv['VAR_NAME']=='Возраст заявителя'].WOE

In [ ]:
o = offset/n

In [ ]:
o

In [ ]:
beta = 0.2

In [ ]:
for kk in final_iv[final_iv['VAR_NAME']=='Возраст заявителя'].WOE:
    #print(kk)
    ll = round(beta*kk+alpha/n,3)
    tt=ll*factor
    score = tt + o
    print(round(score))

In [ ]:
final_iv[final_iv['VAR_NAME']=='Возраст заявителя']

Таким образом получаем следующие скорбаллы:
- Возраст заявителя от - 5 до 3 :10 баллов
- Возраст заявителя от  34 до 43 :7 баллов
- Возраст заявителя от  44 до 905 :6 баллов

минус 5 и 905 - это выбросы, надо было от них избавиться

## Импорт в Excel

In [ ]:
writer = pd.ExcelWriter('Score.xlsx', engine='xlsxwriter')

workbook  = writer.book
worksheet = workbook.add_worksheet('Sample information')
bold = workbook.add_format({'bold': True})
percent_fmt = workbook.add_format({'num_format': '0.00%'})

worksheet.set_column('A:A', 20)
worksheet.set_column('B:B', 15)
worksheet.set_column('C:C', 10)

# Sample
worksheet.write('A2', 'Sample conditions', bold)
worksheet.write('A3', 1)
worksheet.write('A4', 2)
worksheet.write('A5', 3)
worksheet.write('A6', 4)

# Model
worksheet.write('A8', 'Model development', bold)

worksheet.write('A9', 1)
#labels
worksheet.write('C8', 'Bads')
worksheet.write('D8', 'Goods')
worksheet.write('B9', 'Train')
worksheet.write('B10', 'Valid')
worksheet.write('B11', 'Total')

# goods and bads
worksheet.write('C9', y_train.value_counts()[1])
worksheet.write('C10', y_test.value_counts()[1])
worksheet.write('D9', y_train.value_counts()[0])
worksheet.write('D10', y_test.value_counts()[0])
worksheet.write('C11', y.value_counts()[1])
worksheet.write('D11', y.value_counts()[0])

# NPL
worksheet.write('A13', 2)
worksheet.write('B13', 'NPL')
worksheet.write('C13', (y.value_counts()[1]/(y.value_counts()[1]+y.value_counts()[0])), percent_fmt)

worksheet.write('A16', 3)
worksheet.write('C15', 'Gini')
worksheet.write('B16', 'Train')
worksheet.write('B17', 'Valid')
worksheet.write('B18', 'CV Scores')
worksheet.write('C18', str([round(sc, 2) for sc in scores]))

worksheet.write('C16', round(2*roc_auc_score(y_train, clf_lr.predict_proba(X_train)[:,1]) - 1, 3))
worksheet.write('C17', round(2*roc_auc_score(y_test, clf_lr.predict_proba(X_test)[:,1]) - 1, 3))

# Regreesion coefs
feat.to_excel(writer, sheet_name='Regression coefficients', index=False)
worksheet2 = writer.sheets['Regression coefficients']
worksheet2.set_column('A:A', 15)
worksheet2.set_column('B:B', 50)

#WOE

ivs[['VAR_NAME', 'Variable range', 'WOE', 'COUNT', 'WOE_group']].to_excel(writer, sheet_name='WOE', index=False)
worksheet3 = writer.sheets['WOE']
worksheet3.set_column('A:A', 50)
worksheet3.set_column('B:B', 60)
worksheet3.set_column('C:C', 30)
worksheet3.set_column('D:D', 20)
worksheet3.set_column('E:E', 12)
for num, i in enumerate([x.replace('WOE_','') for x in X_train.columns]):
        ev = iv_df[iv_df.VAR_NAME==i]
        ev.reset_index(inplace=True)
        worksheet3.insert_image('G{}'.format(num*34+1), '{}.png'.format(i))

df3.to_excel(writer, sheet_name='Data', index=False)

table.to_excel(writer, sheet_name='Scores by buckets', header = True, index = True)
worksheet4 = writer.sheets['Scores by buckets']
worksheet4.set_column('A:A', 20)
worksheet4.insert_image('J1', 'score_distribution.png')
Ginis.to_excel(writer, sheet_name='Gini distribution', header = True, index = True)
worksheet5 = writer.sheets['Gini distribution']
worksheet5.insert_image('E1', 'gini_stability.png')
writer.save()